In [ ]:
# Bhawesh Pandit

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        paths.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# unpack data to working folder 
import zipfile
for file in paths:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('/kaggle/working')

# Imports

In [ ]:
import warnings
import numpy as np 
import pandas as pd 
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from haversine import haversine
from sklearn.model_selection import train_test_split

tqdm.pandas()
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data Splitting: (Max 0 Points)

In [ ]:
df = pd.read_csv("/kaggle/working/train.csv")

# EDA on Training Data (Max 3 Points)

1. Identify and Handle Anomalies:

	- Analyze the daily trip count and identify two anomalous points in the data (e.g., unusually high or low demand).
	- Create a new categorical feature `is_anomaly`, where:
	- 1 indicates days with abnormal demand.
	- 0 indicates regular days.

2. Extract useful temporal features from the `pickup_datetime` column:
	- `pickup_day_of_year` (ordinal day of the year)
	- `pickup_day_of_week` (0=Monday, 6=Sunday)
	- `pickup_hour_of_day` (hour of the day)
	These features will help the model capture time-based patterns.

3. Analyze Traffic Congestion:
	Calculate the average speed (distance divided by trip duration) for trips based on the day of the week and hour of the day.
	Use this analysis to create a categorical feature for traffic congestion levels, with categories such as:
	- 0: Light traffic
	- 1: Heavy traffic

4. Passenger Count Analysis:
	- Explore the passenger_count column to identify:
	- Common values (e.g., 1 or 2 passengers).
	- Outliers (e.g., trips with zero or unusually high passenger counts).
	- Decide whether to retain or remove these outliers and explain your choice.

5. Using `pickup_longitude`, `pickup_latitude`, `dropoff_longitude`, and `dropoff_latitude`, compute the straight-line distance (trip_distance) using the `haversine` library.

6. Airport Proximity Features: Create binary features to indicate whether the trip started or ended at one of NYC’s major airports:

7. Create binary features indicating whether the trip started or ended within the NYC boundaries. Use bounding box limits or geospatial libraries to define NYC. Use boundaries from previous HW.

8. Analyze the NYC map and improve the distance calculation.

9. Handle Geospatial Outliers: Identify and remove outliers in the test and training datasets based on unrealistic coordinates (e.g., trips starting in the ocean or far outside NYC).

10. Analyze and Clean Trip Duration:
	- Plot the distribution of trip_duration and identify outliers (e.g., trips that are excessively short or long).
	- Remove these outliers if they significantly distort the data, and justify your decision.



### 1. Identify and Handle Anomalies:

	- Analyze the daily trip count and identify two anomalous points in the data (e.g., unusually high or low demand).
	- Create a new categorical feature `is_anomaly`, where:
	- 1 indicates days with abnormal demand.
	- 0 indicates regular days.

In [ ]:
# Convert pickup_datetime to datetime format and convert to date pickup_date
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['pickup_date'] = df['pickup_datetime'].dt.date

# Calculate daily trip counts
daily_trips = df.groupby('pickup_date').size().reset_index(name='trip_count')

# Calculate mean and standard deviation
mean_trips = daily_trips['trip_count'].mean()
std_trips = daily_trips['trip_count'].std()

# this is my threshold I checked 1,2,3 and choose 1.5
threshold = 1.5

lower_bound = mean_trips - threshold * std_trips
upper_bound = mean_trips + threshold * std_trips
print(lower_bound, upper_bound)
high_anomaly = daily_trips[daily_trips['trip_count'] > upper_bound]
low_anomaly = daily_trips[daily_trips['trip_count'] < lower_bound]

# Combine high and low anomalies
anomalous_days = pd.concat([high_anomaly, low_anomaly])

df['is_anomaly'] = 0
df.loc[df['pickup_date'].isin(anomalous_days['pickup_date']), 'is_anomaly'] = 1

# Visualize the results
plt.figure(figsize=(12, 6))
plt.plot(daily_trips['pickup_date'], daily_trips['trip_count'], label='Daily trip cout')
plt.axhline(y=lower_bound, color='g', linestyle='--', label='Upper Threshold')
plt.axhline(y=upper_bound,  color='g', linestyle='--', label='Lower Threshold')
plt.scatter(anomalous_days['pickup_date'], anomalous_days['trip_count'], color='red', label='Anomalies')
plt.title('1. Identify and Handle Anomalies')
plt.xlabel('Date')
plt.ylabel('Daily trip count')
plt.legend()
plt.show()

### 2. Extract useful temporal features from the `pickup_datetime` column:
	- `pickup_day_of_year` (ordinal day of the year)
	- `pickup_day_of_week` (0=Monday, 6=Sunday)
	- `pickup_hour_of_day` (hour of the day)
	These features will help the model capture time-based patterns.


In [ ]:
df['pickup_day_of_year'] = df['pickup_datetime'].dt.year
df['pickup_day_of_week'] = df['pickup_datetime'].dt.weekday  # 0=Monday, 6=Sunday
df['pickup_hour_of_day'] = df['pickup_datetime'].dt.hour

df.head()

### 5. Using `pickup_longitude`, `pickup_latitude`, `dropoff_longitude`, and `dropoff_latitude`, compute the straight-line distance (trip_distance) using the `haversine` library.


In [ ]:
def vec_haversine(row):
    return haversine((row['pickup_latitude'], row['pickup_longitude']),(row['dropoff_latitude'], row['dropoff_longitude']))

df['trip_distance'] = df.progress_apply(vec_haversine, axis=1)

### 3. Analyze Traffic Congestion:
	Calculate the average speed (distance divided by trip duration) for trips based on the day of the week and hour of the day.
	Use this analysis to create a categorical feature for traffic congestion levels, with categories such as:
	- 0: Light traffic
	- 1: Heavy traffic

In [ ]:
# Calculate speed (distance in km / duration in hours)
df['speed'] = df['trip_distance'] / (df['trip_duration'] / 3600)

# Calculate average speed by day of the week and hour
df_avg = df.groupby(['pickup_day_of_week', 'pickup_hour_of_day'])['speed'].mean().reset_index()

# Visualize to make decision 
fig, ax = plt.subplots(figsize=(10, 10))
df_temp = df_avg.pivot(index='pickup_hour_of_day', columns='pickup_day_of_week', values='speed')
sns.heatmap(df_temp, annot=True, ax=ax)
plt.show()

In [ ]:
df_avg['traffic_congestion'] = 0
# Based on heatmap I choosed this: 
df_avg.loc[(((8 <= df_avg['pickup_hour_of_day']) & (df_avg['pickup_hour_of_day'] <= 18)) & ((1 <= df_avg['pickup_day_of_week']) & (df_avg['pickup_day_of_week'] <= 4))), 'traffic_congestion'] = 1
df_grb = df_avg.groupby(['pickup_day_of_week', 'pickup_hour_of_day'])[['traffic_congestion']].max().reset_index()
df_temp = df_grb.pivot(index='pickup_hour_of_day', columns='pickup_day_of_week', values='traffic_congestion')

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df_temp, annot=True, ax=ax)
plt.show()

In [ ]:
# My trafic condition convert to train dataframe
df['traffic_congestion'] = 0
df.loc[(((8 <= df['pickup_hour_of_day']) & (df['pickup_hour_of_day'] <= 18)) & ((1 <= df['pickup_day_of_week']) & (df['pickup_day_of_week'] <= 4))), 'traffic_congestion'] = 1

df.head()

### 4. Passenger Count Analysis:
	- Explore the passenger_count column to identify:
	- Common values (e.g., 1 or 2 passengers).
	- Outliers (e.g., trips with zero or unusually high passenger counts).
	- Decide whether to retain or remove these outliers and explain your choice.


In [ ]:
# Check unique values and their counts
passenger_counts = df['passenger_count'].value_counts().sort_index()
print(passenger_counts)

In [ ]:
# I decided to remove 7, 8, 9 based on the counts very small, and I keep 0 because it could be realistic scenario
# Filter for common values and decide outliers
outliers = [7, 8, 9]
df = df[~df['passenger_count'].isin(outliers)]

df.head()


### 6. Airport Proximity Features: Create binary features to indicate whether the trip started or ended at one of NYC’s major airports:


In [ ]:
# Define NYC airports coordinates (approximate):
airports = [
    (40.6413, -73.7781), # JFK
    (40.7769, -73.8740), # LaGuardia
    (40.6895, -74.1745) # Newark
]

def is_near_airport(lat, lon, airport_coords, radius=1.5):
    return haversine((lat, lon), airport_coords) <= radius

# Apply to each row to check if the pickup or dropoff is near an airport
df['pickup_near_airport'] = df.progress_apply(lambda x: any(is_near_airport(x['pickup_latitude'], x['pickup_longitude'], airport) for airport in airports), axis=1)
df['dropoff_near_airport'] = df.progress_apply(lambda x: any(is_near_airport(x['dropoff_latitude'], x['dropoff_longitude'], airport) for airport in airports), axis=1)

df.head()


### 7. Create binary features indicating whether the trip started or ended within the NYC boundaries. Use bounding box limits or geospatial libraries to define NYC. Use boundaries from previous HW.


In [ ]:
manhattan_bounds = {
    'lat_min': 40.70,  
    'lat_max': 40.88, 
    'lon_min': -74.02, 
    'lon_max': -73.90
}

def is_within_nyc(lat, lon):
    return (manhattan_bounds['lat_min'] <= lat <= manhattan_bounds['lat_max'] and manhattan_bounds['lon_min'] <= lon <= manhattan_bounds['lon_max'])

df['pickup_in_nyc'] = df.progress_apply(lambda x: is_within_nyc(x['pickup_latitude'], x['pickup_longitude']), axis=1)
df['dropoff_in_nyc'] = df.progress_apply(lambda x: is_within_nyc(x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

df.head()


### 8. Analyze the NYC map and improve the distance calculation.


In [ ]:
!pip install osmnx
import osmnx as ox

G = ox.graph_from_place('New York, USA', network_type='drive')

In [ ]:
'''
def vec_osmnx(row):
    try:
        pickup_node = ox.distance.nearest_nodes(G, row['pickup_longitude'], row['pickup_latitude'])
        dropoff_node = ox.distance.nearest_nodes(G, row['dropoff_longitude'], row['dropoff_latitude'])
        route_nodes = ox.shortest_path(G, pickup_node, dropoff_node, weight="length", cpus=None)
        edge_lengths = ox.routing.route_to_gdf(G, route_nodes)
        return sum(edge_lengths["length"])/1000
    except:
        return row['trip_distance']

df['trip_distance_ox'] = df.progress_apply(vec_osmnx, axis=1)

# It took too long, I decided not to use it.
'''

In [ ]:
%%time

row = df.loc[df['trip_distance'] == df['trip_distance'].max()]
pickup_longitude = row.iloc[0]['pickup_longitude']
pickup_latitude = row.iloc[0]['pickup_latitude']
dropoff_longitude = row.iloc[0]['dropoff_longitude']
dropoff_latitude = row.iloc[0]['dropoff_latitude']

pickup_node = ox.distance.nearest_nodes(G, pickup_longitude, pickup_latitude)
dropoff_node = ox.distance.nearest_nodes(G, dropoff_longitude, dropoff_latitude)
route_nodes = ox.shortest_path(G, pickup_node, dropoff_node, weight="length", cpus=None)
edge_lengths = ox.routing.route_to_gdf(G, route_nodes)

print(row.iloc[0].id, row.iloc[0].trip_distance, sum(edge_lengths["length"]))
fig, ax = ox.plot_graph_route(G, route_nodes, route_color="r", route_linewidth=6, node_size=0)
plt.show()


### 9. Handle Geospatial Outliers: Identify and remove outliers in the test and training datasets based on unrealistic coordinates (e.g., trips starting in the ocean or far outside NYC).

In [ ]:
blen = df.shape[0]
df = df[df['pickup_in_nyc'] & df['dropoff_in_nyc']]

print(f'Removed: {blen-df.shape[0]}')
df.head()

### 10. Analyze and Clean Trip Duration:
	- Plot the distribution of trip_duration and identify outliers (e.g., trips that are excessively short or long).
	- Remove these outliers if they significantly distort the data, and justify your decision.


In [ ]:
df['trip_duration'] = df['trip_duration'].progress_apply(np.log1p)

plt.hist(df['trip_duration'], bins = 100)
plt.title('log(trip_duration)')
plt.show()

In [ ]:
best_constant = df['trip_duration'].mean()
print(best_constant, np.expm1(best_constant))
min_duration, max_duration = 4, 8
print(np.expm1(min_duration), np.expm1(max_duration))
'''
I decided to choose trip duration between 54s and 8102s
'''
blen = df.shape[0]
df = df[(df['trip_duration'] >= min_duration) & (df['trip_duration'] <= max_duration)]

print(f'Removed: {blen-df.shape[0]}')

In [ ]:
df

# Feature Preprocessing (Max 2 Points)

1. Divide features into numerical and categorical groups and justify your choice:

2. Preprocess Features: 
   
 	For numerical features:

 	 - Handle missing values (if any).
 	 - Normalize or standardize features to ensure consistent scaling.
     
 	For categorical features:

 	 - Encode them using one-hot encoding or ordinal encoding, as appropriate.
  
  Use sklearn.pipeline to streamline preprocessing and model training

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score

numerical_features = [
    'speed',
    'trip_distance',
    'passenger_count',
    'pickup_hour_of_day',
]

categorical_features = [
    'is_anomaly',
    'pickup_day_of_week',
    'traffic_congestion',
    'pickup_near_airport',
    'dropoff_near_airport'
]

df = df[numerical_features+categorical_features+['trip_duration']]
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

X_train = train.drop(columns=['trip_duration'])
y_train = train['trip_duration']
X_test = test.drop(columns=['trip_duration'])
y_test = test['trip_duration']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('scaling', StandardScaler(), numerical_features),
        ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Ridge())
])

model = pipeline.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f"Train RMSE = {mean_squared_error(y_train, model.predict(X_train), squared=False)}")
print(f"Test RMSE = {mean_squared_error(y_test, y_pred, squared=False)}")

# Model Training and Evaluation (Max 4 Points)

## Linear model
1. Use DummyRegressor as a baseline model to predict trip duration.
2. Use LinearRegression, Ridge, and Lasso Regression from sklearn with Mean Squared Error (MSE) metric to evaluate performance on the test set. For metric evaluation use cross_val_score with `cv=10`
3. Calculate feathure importance for feathures
4. Use Lasso regression for feature selection. Is removing feathure improve your result?
5. Tune Hyperparameters (if needed) using GridSearchCV or RandomizedSearchCV.

## DecisionTreeRegressor
1. Use DecisionTreeRegressor with Mean Squared Error (MSE) metric to evaluate performance on the test set
2. Tune Hyperparameters (if needed) using GridSearchCV or RandomizedSearchCV.
3. Calculate feathure importance for feathures

## RandomForest
1. Use RandomForestRegressor with Mean Squared Error (MSE) metric to evaluate performance on the test set
2. Tune Hyperparameters (if needed) using GridSearchCV or RandomizedSearchCV.
3. Calculate feathure importance for feathures

## Linear model

### 1. Use DummyRegressor as a baseline model to predict trip duration.


In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DummyRegressor(strategy="mean"))
])
dummy_model = pipeline.fit(X_train, y_train)
y_pred = dummy_model.predict(X_test)

print(f"Dummy Regressor RMSE = {mean_squared_error(y_test, y_pred, squared=False)}")

### 2. Use LinearRegression, Ridge, and Lasso Regression from sklearn with Mean Squared Error (MSE) metric to evaluate performance on the test set. For metric evaluation use cross_val_score with `cv=10`

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
    mse = -scores.mean()
    print(f"{name} MSE: {mse:.4f}")

### 3. Calculate feature importance for features

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

selected_features = pd.Series(lr.coef_, index=X_train.columns).sort_values(ascending=False)

print("Linear Regression Feature Importance:")
print(selected_features)

In [ ]:
plt.rcParams['figure.figsize'] = (8.0, 8.0)
selected_features.plot(kind = "barh")
plt.title("Feature importance using Linear Regression Model")
plt.show()

In [ ]:
# I decided  to remove dropoff_near_airport and pickup_near_airport
categorical_features = ['is_anomaly', 'pickup_day_of_week', 'traffic_congestion']
preprocessor = ColumnTransformer(
    transformers=[
        ('scaling', StandardScaler(), numerical_features),
        ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
    mse = -scores.mean()
    print(f"{name} with selected features MSE: {mse:.4f}")

### 4. Use Lasso regression for feature selection. Is removing feathure improve your result?

In [ ]:
lasso = Lasso(alpha=1e-4)
lasso.fit(X_train, y_train)

selected_features = pd.Series(lasso.coef_, index=X_train.columns).sort_values(ascending=False)

print("Lasso Feature Importance:")
print(selected_features)

In [ ]:
'''
Linear and Lasso Regression are I get around same result so I decide to not to use
'''

### 5. Tune Hyperparameters (if needed) using GridSearchCV or RandomizedSearchCV.

In [ ]:
param_grid = {'alpha': np.logspace(-2, 3, 10)}
grid = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

print(f"Best parameters for Ridge: {grid.best_params_['alpha']:.3f}")
print(f"Best MSE: {-grid.best_score_:.4f}")

## DecisionTreeRegressor
### 1. Use DecisionTreeRegressor with Mean Squared Error (MSE) metric to evaluate performance on the test set

In [ ]:
%%time
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state = 42)
dtr.fit(X_train, y_train)
mse = mean_squared_error(y_test, dtr.predict(X_test))

print(f"Decision Tree Regressor MSE: {mse:.4f}")

### 2. Tune Hyperparameters (if needed) using GridSearchCV or RandomizedSearchCV.

In [ ]:
%%time
param_grid = {
    'max_depth': [5, 10],
    'min_samples_leaf': [2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
grid_dtr = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_dtr.fit(X_train, y_train)

print("Best parameters for Random Forest:", grid_dtr.best_params_)
print(f"Best MSE: {-grid_dtr.best_score_:.4f}")

### 3. Calculate feathure importance for feathures

In [ ]:
best_dtr = grid_dtr.best_estimator_
importances = best_dtr.feature_importances_

selected_features = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
print("Feature Importance for Tuned Decision Tree:")
print(selected_features)

## RandomForest
### 1. Use RandomForestRegressor with Mean Squared Error (MSE) metric to evaluate performance on the test set


In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=42, max_features=3)
rf.fit(X_train, y_train)

mse = mean_squared_error(y_test, rf.predict(X_test))

print(f"Random Forest MSE: {mse:.4}")

### 2. Tune Hyperparameters (if needed) using GridSearchCV or RandomizedSearchCV.

In [ ]:
%%time
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_leaf': [2, 4]
}

grid_rf = GridSearchCV(RandomForestRegressor(random_state = 42), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_rf.fit(X_train, y_train)

print("Best parameters for Random Forest:", grid_rf.best_params_)
print(f"Best MSE: {-grid_rf.best_score_:.4f}")

### 3. Calculate feathure importance for feathures

In [ ]:
best_rf = grid_rf.best_estimator_
importances = best_rf.feature_importances_

selected_features = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
print("Feature Importance from Random Forest:")
print(selected_features)

# Conclusion (Max 1 Points)

Select best model, explain results

In [ ]:
dtr = DecisionTreeRegressor(**grid_dtr.best_params_)
dtr.fit(X_train, y_train)
mse = mean_squared_error(y_test, dtr.predict(X_test))

print(f"Decision Tree MSE: {mse:.4f}")

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_dtr)
])

scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
mse = -scores.mean()
tmse = mean_squared_error(y_test, pipeline['model'].predict(X_test))

print(f"Decision Tree MSE with selected features: {mse:.4f}")
print(f"Decision Tree MSE on Test: {tmse:.4f}")

In [ ]:
rf = RandomForestRegressor(**grid_rf.best_params_)
rf.fit(X_train, y_train)

mse = mean_squared_error(y_test, rf.predict(X_test))

print(f"Random Forest MSE: {mse:.4f}")

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_rf)
])

scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
mse = -scores.mean()

tmse = mean_squared_error(y_test, pipeline['model'].predict(X_test))
print(f"Random Forest MSE with selected features: {mse:.4f}")
print(f"Random Forest MSE on Test: {tmse:.4f}")

In [ ]:
#Linear Regression MSE with selected features: 0.094
#Ridge MSE with selected features: 0.094
#Lasso MSE with selected features: 0.44
#Decision Tree MSE: 0.004
#Random Forest MSE: 0.003


**I'll choose the Decision Tree instead of the Random Forest. While the Random Forest offers the best performance, its training time is significantly longer, and it is more prone to overfitting in the long run**
